# Урок 3

В данном уроке мы попробуем запустить наше DTW уже на реальном звуке. Для начала это будут простые слова "да" и "нет". Имеется небольшая база с файлами различных вариаций произнесения этих слов. Часть из них (эталоны) будут использованны для построения графа. Для остальных же (записей) будет поочередно запущен DTW алгорим для определения ближайшего к ним файла из эталонов.

MFCC признаки мы уже посчитали и сохранили в формате ark в файлах etalons_mfcc.txtftr и records_mfcc.txtftr соответственно.

Ранее, наш граф был способен идти только вровень, либо сжимать запись (оставаться в том же узле графа) относительно кадров эталона. Но необходимо еще уметь и растягивать запись. Для этого нужно ввести дополнительные переходы через один и два состояния для узлов графа.

<br>
<img src="graph.png">
<br>

<b>Задание 1:</b>
Добавить для узлов графа дополнительные переходы через один и два состояния (нулевой узел должен остаться прежним).

In [5]:
import numpy as np
import FtrFile


class State:
    def __init__(self, ftr, idx):
        self.ftr = ftr           # вектор признаков узла 
        self.isFinal = False     # является ли этот узел финальнвм в слове
        self.word = None         # слово эталона (назначается только для финального узла)       
        self.nextStates = []     # список следующих узлов
        self.idx = idx           # индекс узла
        self.bestToken = None    # лучший токен (по минимуму дистанции) в узле
        self.currentWord = None  # текущее слово эталона
        
        
def load_graph(rxfilename):
    startState = State(None, 0)
    graph = [startState, ]
    stateIdx = 1
    for word, features in FtrFile.FtrDirectoryReader(rxfilename):
        prevState = startState
        for frame in range(features.nSamples):
            state = State(features.readvec(), stateIdx)
            state.currentWord = word           # слово эталона теперь будет храниться в каждом узле
            state.nextStates.append(state)          
            prevState.nextStates.append(state)
            prevState = state
            
            #-----------------------------TODO-----------------------------------
            
            #--------------------------------------------------------------------
            stateIdx += 1
            
        if state:
            state.word = word
            state.isFinal = True
    return graph

Следующий блок кода проверяет ваш граф на некоторые ключевые параметры и записывает в удобном для чтения виде в файл graph.txt. Сравните его с заведомо правильным графом, сохраненным в graph_reference.txt:

In [6]:
def check_graph(graph):
    assert len(graph) > 0, "graph is empty."
    assert graph[0].ftr is None \
        and graph[0].word is None \
        and not graph[0].isFinal, "broken start state in graph."
    idx = 0
    for state in graph:
        assert state.idx == idx
        idx += 1
        assert (state.isFinal and state.word is not None) \
            or (not state.isFinal and state.word is None)


def print_graph(graph):
    with open('graph.txt', 'w') as fn:
        np.set_printoptions(formatter={'float': '{: 0.1f}'.format})
        for state in graph:
            nextStatesIdxs = [s.idx for s in state.nextStates]
            fn.write("State: idx={} word={} isFinal={} nextStatesIdxs={} ftr={} \n".format(
                state.idx, state.word, state.isFinal, nextStatesIdxs, state.ftr))
    print("*** SEE graph.txt ***")
    print("*** END DEBUG. GRAPH ***")

    
etalons = "ark,t:etalons_mfcc.txtftr"
graph = load_graph(etalons)
check_graph(graph)
# Сохранить граф в читабельном виде в файл graph.txt:
print_graph(graph)

*** SEE graph.txt ***
*** END DEBUG. GRAPH ***


Реализованный в прошлом уроке TPA в данном случае будет перебирать все возможные варианты разметки, что приведет к значительному увеличению времени работы нашего DTW. Для решения этой проблемы мы будем отбрасывать "ненужные" токены еще на этапе прохождения по графу. Этим занимаются, так называемые, beam и state prunings.

### state pruning:
В классе State нужно добавить атрибут best_token – ссылку на лучший токен, заканчивающийся в данном стейте на данном кадре записи. После порождения всех токенов за текущий кадр записи, пройдемся по каждому из полученных nextTokens, затем впишем текущий токен в State.best_token (здесь State – это узел, на котором закончился токен), убив предыдущий лучший токен, либо убьем сам токен, если он хуже лучшего на этом узле. За жизнеспособность токена отвечает его атрибут is_alive: True или False соответственно.

После этого необходимо очистить поле best_token у всех узлов графа.

### beam pruning:
Идея состоит в том, чтобы на каждом кадре записи находить плохие токены и откидывать их (token.is_alive = False). 
Плохие – это,очевидно, накопившие слишком большое отклонение от стейтов,по которым они идут. Слишком большое отклонение – это непонятно какое (может токен плохой, может слово слишком длинное, может звук очень плохой – не разобрать).

Поэтому плохость токена считают относительно лучшего токена. Заведем переменную thr_common (обычно её называют beam – ширина луча поиска; у нас это common threshold – “общий порог” – по историческим причинам). И если token.dist > best_token.dist + thr_common, то token плохой и мы его отбросим.

Выкидывая какой-то токен из-за его отклонеиня, мы рискуем тем, что через сколько-то кадров все потомки выживших токенов могут оказаться очень плохими, а только потомки отброшенного токена оказались бы чудо как хороши. То есть, вводя thr_common, мы вводим ошибку.
Поэтому thr_common нужно подобрать так, чтобы скорость сильно выросла, а ошибка выросла незначительно.

<br>
Введение этих методов может привести к тому, что у нас просто не окажется в конце выживших токенов в финальных узлах графа. Для того, чтобы иметь возможность выдавать результат в этом случае, мы введем дополнительный атрибут currentWord у класса State. Теперь в любом узле каждой ветви будет храниться слово соответствующего эталона для этой ветви. 

Тогда в конце работы DTW, если у нас не будет живых финальных токенов, то мы просто выберем лучший из оставшихся и по полю state.currentWord определим слово эталона.

<b>Задание 2:</b> 
- Написать функцию findBest для поиска токена с минимальной дистанцией.
- Реализовать функции для state и beam pruning (здесь нам и может пригодиться функция findBest).
- Разобраться с вычислением WER.

In [7]:
class Token:
    def __init__(self, state, dist=0.0, sentence=""):
        self.state = state
        self.dist = dist
        self.sentence = sentence
        self.alive = True
    

def findBest(Tokens):
    #---------------------------------------TODO---------------------------------  
    
    #-----------------------------------------------------------------------------
    return bestToken


def beamPruning(nextTokens):
    thr_common = 70 # можно менять
    #--------------------------------TODO--------------------------------------
    # 1. Ищем лучший токен из nextTokens с помощью findBest
    # 2. Присваиваем token.aliv значение False, если дистанция этого токена больше, чем
    #    длина лучшего токена + thr_common
    
    #--------------------------------------------------------------------------
    return nextTokens


def statePruning(nextTokens):
    for i in range(len(nextTokens)):
        #--------------------------TODO---------------------------------
        #
        #
        #---------------------------------------------------------------
        
    # Сбросить bestToken на None для всеx узлов графа
    
    #-------------------------------------------------------------------
    return nextTokens


def distance(X, Y):
    result = float(np.sqrt(sum(pow(X - Y, 2))))
    return result


def recognize(features, graph, rec_results):

    print("-" * 23)
    startTime = time.time()
    startState = graph[0]
    activeTokens = [Token(startState), ]
    nextTokens = []

    for frame in range(features.nSamples):
        ftrCurrentFrameRecord = features.readvec()
        for token in activeTokens:
            if token.alive:
                for transitionState in token.state.nextStates:
                    newToken = Token(transitionState, token.dist, token.sentence)
                    newToken.dist += distance(ftrCurrentFrameRecord, transitionState.ftr)
                    nextTokens.append(newToken)
        # state and beam prunings:
        nextTokens = statePruning(nextTokens)         
        nextTokens = beamPruning(nextTokens) 

        activeTokens = nextTokens
        nextTokens = []                                    
        
    # поиск финальных токенов:
    finalTokens = []
    for token in activeTokens:
        if token.state.isFinal and token.alive:
            finalTokens.append(token)

    # если нет финальных, то берем лучший из выживших:
    if len(finalTokens) != 0:
        winToken = findBest(finalTokens)
    else:
        winToken = findBest(activeTokens)
        winToken.state.word = winToken.state.currentWord

    # вывод результата DTW
    print("result: {} ==> {}".format(filename, winToken.state.word))
    endTime = time.time()
    print("time: {} sec".format(round(endTime-startTime, 2)))

    # совпадает ли запись с полученным эталоном:  
    record_word = filename.split('_')[0]
    etalon_word = winToken.state.word.split('_')[0]
    rec_results.append(etalon_word == record_word)

    return frame

Теперь запустим нашу программу.

In [8]:
import time

etalons = "ark,t:etalons_mfcc.txtftr"
records = "ark,t:records_mfcc.txtftr"

rec_results = []  # переменная для подсчета точности распознавания

s_time = time.time()
numbFrame = 0     # счетчик общего количества кадров для расчета RTF

graph = load_graph(etalons)

for filename, features in FtrFile.FtrDirectoryReader(records):
    frame = recognize(features, graph, rec_results)
    numbFrame += frame

print("-" * 23)
print("WER is: {}".format(round(1 - sum(rec_results)/len(rec_results), 3)))
e_time = time.time()
time = e_time-s_time
minut = int(time/60)
second = int(time-minut*60)
print("Total time: {} min {} sec".format(minut, second))
rtf = round(time/(numbFrame*0.01), 2)
print("RTF is: {}".format(rtf))

-----------------------
result: da_06 ==> da_02
time: 2.37 sec
-----------------------
result: da_07 ==> da_02
time: 5.09 sec
-----------------------
result: da_08 ==> da_04
time: 3.58 sec
-----------------------
result: da_09 ==> da_02
time: 3.44 sec
-----------------------
result: da_10 ==> da_04
time: 4.56 sec
-----------------------
result: da_11 ==> da_02
time: 4.17 sec
-----------------------
result: da_12 ==> da_05
time: 3.12 sec
-----------------------
result: da_13 ==> da_02
time: 3.02 sec
-----------------------
result: da_14 ==> da_02
time: 2.81 sec
-----------------------
result: da_15 ==> da_02
time: 5.24 sec
-----------------------
result: da_16 ==> da_02
time: 2.38 sec
-----------------------
result: da_17 ==> da_05
time: 4.06 sec
-----------------------
result: da_18 ==> da_05
time: 2.15 sec
-----------------------
result: da_19 ==> da_05
time: 4.04 sec
-----------------------
result: net_06 ==> da_02
time: 3.12 sec
-----------------------
result: net_07 ==> net_04
time

<b>Задание 3:</b> Подбирите значение порога thr_common и количество дополнительных переходов для узлов так, чтобы получить минимально возможно значение WER для данной базы.